# Run detectron2 on jpeg's of papers

Assumes you have:
1. Run the ocr_and_image_processing_batch.py file to generate the JPEGs
1. Run the `pull_check_makesense.ipynb` file to create annotation files, and then
1. Annotated on MakeSense.ai and downloaded those annotations
1. Run the `process_annotations.py` file to generate the yolo-style-XML anntation files

Also, specific for this version of detectron2, you need the weights trained on [the PubLayNet dataset](https://github.com/JPLeoRX/detectron2-publaynet).  You can do this in the command line with:

```
wget https://raw.githubusercontent.com/hpanwar08/detectron2/master/configs/DLA_mask_rcnn_R_101_FPN_3x.yaml
wget https://raw.githubusercontent.com/hpanwar08/detectron2/master/configs/DLA_mask_rcnn_R_50_FPN_3x.yaml
wget https://raw.githubusercontent.com/hpanwar08/detectron2/master/configs/DLA_mask_rcnn_X_101_32x8d_FPN_3x.yaml
wget https://raw.githubusercontent.com/facebookresearch/detectron2/main/configs/Base-RCNN-FPN.yaml
```

Make sure you save these weights in a folder where you remember!

In [1]:
detectron2_weights_dir = '/Users/jnaiman/Dropbox/wwt_image_extraction/FigureLocalization/BenchMarks/model_params/detectron2/'

In [2]:
import argparse
import glob
import multiprocessing as mp
import numpy as np
import os
import tempfile
import time
import warnings
import cv2
import tqdm

from detectron2.config import get_cfg
from detectron2.data.detection_utils import read_image
from detectron2.utils.logger import setup_logger

In [3]:
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

Rename classes for the PubLayNet dataset:

In [4]:
MetadataCatalog.get("dla_val").thing_classes = ['text', 'title', 'list', 'table', 'figure']

General config file set up:

In [5]:
def setup_cfg(configfile, weightsfile, processingType='cpu', confidence_threshold=0.5):
    # load config from file and command-line arguments
    cfg = get_cfg()
    # To use demo for Panoptic-DeepLab, please uncomment the following two lines.
    # from detectron2.projects.panoptic_deeplab import add_panoptic_deeplab_config  # noqa
    # add_panoptic_deeplab_config(cfg)
    cfg.merge_from_file(configfile)
    argsList = ['MODEL.WEIGHTS',weightsfile, 'MODEL.DEVICE',processingType]
    #print(argsList)
    cfg.merge_from_list(argsList)
    # Set score_threshold for builtin models
    cfg.MODEL.RETINANET.SCORE_THRESH_TEST = confidence_threshold
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = confidence_threshold
    cfg.MODEL.PANOPTIC_FPN.COMBINE.INSTANCES_CONFIDENCE_THRESH = confidence_threshold
    cfg.freeze()
    return cfg